<a href="https://colab.research.google.com/github/pachterlab/kallistobustools/blob/master/notebooks/kb_QC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to single-cell RNA-seq I: pre-processing and quality control

This notebook demonstrates the use of the kallisto and bustools programs for pre-processing single-cell RNA-seq data. It streams in 1 million *C. elegans* reads, pseudoaligns them, and produces a *cells x genes* count matrix in about a minute. The notebook then performs some basic QC. It expands on a notebook prepared by Sina Booeshaghi for the Genome Informatics 2019 meeting, where he ran it in under 60 seconds during a 1 minute "lightning talk".

The notebook was written by A. Sina Booeshaghi and Lior Pachter. If you use the methods in this notebook for your analysis please cite the following publication, on which it is based:

* Melsted, P., Booeshaghi, A.S. et al. Modular and efficient pre-processing of single-cell RNA-seq. bioRxiv (2019). doi:10.1101/673285

## Setup

In [0]:
# This is  used to time the running of the notebook
start_time <- Sys.time()

### Install R packages

In [4]:
install.packages(c("rsvd", "ggpointdensity"),Ncpus = 2)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [5]:
library(Matrix)
library(rsvd)
library(ggplot2) # Tidyverse is pre-installed, yay!
library(dplyr)
library(tidyr)
library(ggpointdensity)
theme_set(theme_bw())


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack




### Install kb-python (includes kallisto and bustools)

In [0]:
system("pip3 install kb-python")

## Download required files

In [7]:
system("which kb", intern = TRUE)

[1] "/usr/local/bin/kb"

In [0]:
# The quantification of single-cell RNA-seq with kallisto requires an index. 
# Indices are species specific and can be generated or downloaded directly with `kb`. 
# Here we download a pre-made index for C. elegans (the idx.idx file) along with an auxillary file (t2g.txt) 
# that describes the relationship between transcripts and genes.
download.file("https://caltech.box.com/shared/static/82yv415pkbdixhzi55qac1htiaph9ng4.idx",
destfile = "idx.idx")
download.file("https://caltech.box.com/shared/static/cflxji16171skf3syzm8scoxkcvbl97x.txt",
destfile = "t2g.txt")

## Pseudoalignment and counting

In this notebook we pseudoalign 1 million *C. elegans* reads and count UMIs to produce a *cells x genes* matrix. These are located at XXX and instead of being downloaded, are streamed directly to the Google Colab notebook for quantification. 
See [this blog post](https://sinabooeshaghi.com/2019/07/09/fasterq-to-count-matrices-for-single-cell-rna-seq/) for more details on how the streaming works.

The data consists of a subset of reads from [GSE126954](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE126954) described in the paper:

* Packer, J., Zhu, Q. et al. [A lineage-resolved molecular atlas of C. elegans embryogenesis at single-cell resolution](https://science.sciencemag.org/content/365/6459/eaax1971/tab-e-letters). Science (2019). doi:10.1126/science.aax1971

### Run kallisto and bustools

In [9]:
system("kb count -i idx.idx -g t2g.txt --overwrite -t 2 -x 10xv2 https://caltech.box.com/shared/static/fh81mkceb8ydwma3tlrqfgq22z4kc4nt.gz https://caltech.box.com/shared/static/ycxkluj5my7g3wiwhyq3vhv71mw5gmj5.gz",
intern = TRUE)

character(0)

## Basic QC



### Represent the cells in 2D

In [0]:
mat <- readMM("/content/counts_unfiltered/cells_x_genes.mtx")

In [0]:
# Convert to dgCMatrix
mat <- as(mat, "dgCMatrix")

In [12]:
dim(mat)

[1] 95372 22113

In [0]:
pca_res <- rpca(mat, k = 2) # scales and centers by default

In [0]:
colnames(pca_res$x) <- paste0("PC", 1:15)
plot(pca_res$x[,1:2])

In [0]:
colnames(pca_res$x) <- paste0("PC", 1:15)
options(repr.plot.width=12, repr.plot.height=12)
pairs(pca_res$x[,1:4])

Here cells are in rows and genes are in columns, while usually in single cell analyses, cells are in columns and genes are in rows. Here most "cells" are empty droplets.

### Test for library saturation

### Examine the knee plot

## Remove empty droplets

A common way to remove empty droplets is to find the inflection point on the knee plot.

In [0]:
tot_counts <- rowSums(mat)
df <- tibble(total = tot_counts,
             rank = row_number(desc(total))) %>%
      distinct() %>%
      arrange(rank)

In [0]:
options(repr.plot.width=9, repr.plot.height=6)
ggplot(df, aes(total, rank)) +
  geom_path() +
  scale_x_log10() + scale_y_log10() + annotation_logticks() +
  labs(y = "Barcode rank", x = "Total UMI count")

Crappy data, can't see clear inflection point, well, but this is only a subset of the reads. Probably a threshold of at least 30 UMI would work here.

In [0]:
mat <- mat[rowSums(mat) > 30, colSums(mat) > 0] # Also remove 0 count genes
dim(mat)

In [0]:
df2 <- tibble(nCount = rowSums(mat),
              nGene = rowSums(mat > 0))

In [0]:
df2 %>%
  pivot_longer(everything()) %>%
  ggplot(aes(name, value)) +
  geom_violin() +
  geom_jitter(size = 0.5, alpha = 0.3) +
  scale_y_log10() +
  labs(x = "")

In [0]:
ggplot(df2, aes(nCount, nGene)) +
  geom_pointdensity(size = 0.5, alpha = 0.3) +
  scale_color_viridis_c() +
  scale_x_log10() + scale_y_log10() + annotation_logticks() +
  labs(x = "Total UMI counts", y = "Number of genes detected")

### Exercises

#### Exploring PCA subspaces

In [0]:
# Log normalize, the Seurat way
norm_mat <- log1p(sweep(mat, 1, df2$nCount, FUN = "/") * 1e4)

In [0]:
{pca_res <- rpca(norm_mat, k = 15) # scales and centers by default
ggscreeplot(pca_res)}

In [0]:
colnames(pca_res$x) <- paste0("PC", 1:15)

In [0]:
options(repr.plot.width=12, repr.plot.height=12)
pairs(pca_res$x[,1:4])

## Discussion

This notebook has demonstrated the pre-processing required for single-cell RNA-seq analysis. `kb` is used to pseudoalign reads and to generate a *cells x genes* matrix. Following generation of a matrix, basic QC helps to assess the quality of the data.

In [0]:
# Running time of the notebook
Sys.time() - start_time